In [1]:
k=open('Ch_trainfile_Sentiment_3000.txt','r',encoding='utf-8').readlines()
labels=[]
strings=[]
for kk in k:
    segments=kk.split()
    try:
        int(segments[0])
        labels.append(segments[0])
        str1=''.join(segments[1:len(segments)])
        strings.append(str1)
    except ValueError:
        str1=segments[0]
        strings[-1]=strings[-1]+str1

In [2]:
from collections import defaultdict
import os
import re
import jieba
import codecs

"""
1. 文本切割
"""
# sentence='这样的酒店配这样的价格还算不错'
def sent2word(sentence):
    """
    Segment a sentence to words
    Delete stopwords
    """
    segList = jieba.cut(sentence)
    segResult = []
    for w in segList:
        segResult.append(w)

    fd = open('stopword.txt','r',encoding='utf-8').readlines()
    newSent = []
    stopwords=[s[:-1] for s in fd]
    for word in segResult:
        if word in stopwords:
#             print ("stopword: %s" % word)
            continue
        else:
            newSent.append(word)
    if not newSent:
        newSent=segResult
    return newSent

In [3]:
texts=[]
for i in strings:
    texts.append(' '.join(sent2word(i)))

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/jm/88rqwfqs02l30zlfmtxtc9g80000gn/T/jieba.cache
Loading model cost 16.891 seconds.
Prefix dict has been built succesfully.


In [6]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(texts)
X_train_counts.shape

(2987, 2666)

In [8]:
count_vect.vocabulary_.get('商场')

930

In [9]:
ngram_count_vect = CountVectorizer(ngram_range=(1, 5))
XX_train_counts = ngram_count_vect.fit_transform(texts)
XX_train_counts.shape

(2987, 12421)

In [11]:
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

(2987, 2666)

In [12]:
from sklearn.pipeline import Pipeline

In [13]:
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC

In [15]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [93]:
text_clf = Pipeline([('vect', TfidfVectorizer()),
                     ('clf', LinearSVC()),
])

In [94]:
text_clf.fit(texts[:int(len(texts)*0.8)], labels[:int(len(texts)*0.8)])

Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
  ...ax_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0))])

In [100]:
# try one
predicted = text_clf.predict(texts[int(len(texts)*0.8):])

In [101]:
# try one
correct=0
for id,label in enumerate(labels[int(len(texts)*0.8):]):
    if label==predicted[id]:
        correct+=1
print('accuracy:',(correct/(id+1)))

accuracy: 0.7023411371237458


In [97]:
from sklearn.model_selection import cross_val_score

In [98]:
scores = cross_val_score(text_clf, texts, labels, cv=5)

/usr/local/lib/python3.6/site-packages/sklearn/model_selection/_split.py:605: Warning: The least populated class in y has only 3 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


In [99]:
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.69 (+/- 0.04)


In [102]:
def choose_training_id(p):
    ids=[]
    for i in range(len(texts)):
        rand=random.uniform(0, 1)
        if rand in p:
            ids.append(p.index(rand))
        else:
            ids.append(bisect.bisect_left(p, rand))
    return ids
def update_possibility(p,pred_labels,tested_labels):
    wrong=[]
    correct=[]
    # test 90 examples by classifier trained by 10 examples
    for id,x in enumerate(zip(tested_labels,pred_labels)):
        if x[0]==x[1]:
            correct.append(id)
        else:
            wrong.append(id)
    # record the possibility of the wrongly classified examples 
    error=[p[e] for e in wrong]
    # count total error rate
    error=sum(error)
    # calculate beta
    beta=math.sqrt( abs(error/(1-error)))
    # correct examples will be multiplied by beta
    # incorrect examples will be divided by beta
    if error>1e-10:
        for e in wrong:
            p[e]=p[e]/beta
        for c in correct:
            p[e]=p[e]*beta
    # normalize sum of the possibility to be 1
    total=sum(p)
    p=[x/total for x in p]
    sum_p=[]
    # accumulate the normalized possibility
    for i in range(len(p)):
        sum_p.append(sum(p[:i+1]))
    return p,sum_p
def cal_result(testset,gt):
    # use 9 classifiers to estimate examples' type
    # if more than half of classifiers agree with 'some type'
    # then that is the final result
    acc=[]
    sum_result=0
    # print(len(testset),len(testset[0]),len(gt),len(gt[0])) 3 300 300 3
    for i in range(num_trial):
        temp=[]
        for x in zip(testset[i],gt):
            # print(x)
            if x[0]==x[1]:
                temp.append(1)
            else:
                temp.append(0)
        acc.append(temp)
    # print('acc:',len(acc),len(acc[0]),len(testset),len(testset[0]))
    for i in range(len(testset[0])):#300
        temp=0
        for j in range(len(acc)):#3
            temp+=acc[j][i]
        # print(temp,acc[0][j],acc[1][j],acc[2][j],len(acc))
        if temp>int(len(acc)/2):
            sum_result+=1
    return float(sum_result/len(gt))